In [11]:
#pip install kfp==1.8 --no-build-isolation

In [12]:
from kfp import dsl
from kfp import components as comp
from kfp.compiler import Compiler

# Components

## Basic transaction details
transaction ID | timestamp | amount | currency | payment method

In [13]:
def transaction_details(start_date: str, end_date: str) -> dict:
    import uuid
    import datetime
    import random
    import time
    import pandas as pd

    currencies = [
        'USD', 'EUR', 'GBP', 'JPY', 'CHF', 'CAD', 'AUD', 'NZD', 'CNY', 'INR',
        'RUB', 'BRL', 'ZAR', 'SGD', 'HKD', 'MXN', 'NOK', 'SEK', 'DKK', 'PLN'
    ]

    def random_timestamp(start_date_str, end_date_str, date_format='%Y-%m-%d'):
        # Convert date strings to datetime objects
        start_date = datetime.datetime.strptime(start_date_str, date_format)
        end_date = datetime.datetime.strptime(end_date_str, date_format)
        
        # Convert datetime objects to Unix timestamps
        start_timestamp = int(time.mktime(start_date.timetuple()))
        end_timestamp = int(time.mktime(end_date.timetuple()))
        
        # Generate a random timestamp between the start and end timestamps
        random_timestamp = random.randint(start_timestamp, end_timestamp)
        
        # Convert the random timestamp back to a datetime object
        random_date = datetime.datetime.fromtimestamp(random_timestamp)
        
        return random_date
    
    transaction = {
        'transaction_id':str(uuid.uuid4()),
        'timestamp':str(random_timestamp(start_date, end_date)),
        'amount':round(random.uniform(0, 10000), 2),
        'currency':random.choice(currencies),
        'payment_method':random.choice(['CARD', 'TRANSFER'])
        }
    return transaction

comp_transaction_details = comp.create_component_from_func(
    transaction_details,
    packages_to_install=['pandas']
)

## Business names

In [14]:
def business_names() -> list:
    import random
    adjectives = [
        "Advanced", "Innovative", "Creative", "Dynamic", "Efficient", "Global", 
        "Modern", "Smart", "Sustainable", "Unique", "Visionary", "Reliable", 
        "Proactive", "NextGen", "Future"
    ]
    nouns = [
        "Solutions", "Technologies", "Services", "Consulting", "Systems", 
        "Dynamics", "Enterprises", "Ventures", "Concepts", "Designs", "Strategies",
        "Holdings", "Networks", "Group", "Partners"
    ]
    suffixes = [
        "Inc.", "LLC", "Ltd.", "Corp.", "Co.", "Pty Ltd.", "AG", "GmbH", 
        "Associates", "Holdings"
    ]
    
    business_name1 = f"{random.choice(adjectives)} {random.choice(nouns)} {random.choice(suffixes)}"
    business_name2 = f"{random.choice(adjectives)} {random.choice(nouns)} {random.choice(suffixes)}"
    return [business_name1, business_name2]

comp_business_names = comp.create_component_from_func(
    business_names
)

## Parties involved
sender ID | receiver ID | sender name | receiver name

In [15]:
def parties_involved(transaction: dict, business_names: list) -> dict:
    # Possible types on transactions
    # 1. Person to Person
    # 2. Business to Business
    # 3. Person to Business
    # 4. Business to Person
    import random
    from faker import Faker
    import uuid

    fake = Faker()
    type = random.randint(1, 4)
    
    if type == 1:
        sender_name = fake.name()
        receiver_name = fake.name()
    elif type == 2:
        sender_name = business_names[0]
        receiver_name = business_names[1]
    elif type == 3:
        sender_name = fake.name()
        receiver_name = business_names[1]
    elif type == 4:
        sender_name = business_names[0]
        receiver_name = fake.name()

    transaction['sender_id'] = str(uuid.uuid4())
    transaction['sender_name'] = sender_name

    transaction['receiver_id'] = str(uuid.uuid4())
    transaction['receiver_name'] = receiver_name

    transaction['payment_method'] = random.choice(['CARD', 'TRANSFER'])
    
    return transaction

comp_parties_involved = comp.create_component_from_func(
    parties_involved,
    packages_to_install=['faker']
)

In [16]:
def extract_payment_method(parties_info: dict) -> str:
    return parties_info['payment_method']

comp_extract_payment_method = comp.create_component_from_func(
    extract_payment_method
)

## Payment method details
account number(iban) | bank code(bic) or card number | card type 

In [17]:
def bank_details(transaction:dict, country_code:str='EE', bank_code_length:int=8, account_number_length:int=10) -> dict:
    import random
    import string

    def generate_random_iban(country_code, bank_code_length, account_number_length):

        bank_code = ''.join(random.choices(string.digits, k=bank_code_length))
        account_number = ''.join(random.choices(string.digits, k=account_number_length))
        check_digits = ''.join(random.choices(string.digits, k=2))
        
        iban = f"{country_code}{check_digits} {bank_code} {account_number}"
        return iban

    def generate_random_bic():
        # Generate random BIC components
        bank_code = ''.join(random.choices(string.ascii_uppercase, k=4))
        country_code = ''.join(random.choices(string.ascii_uppercase, k=2))
        location_code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=2))
        branch_code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))
        
        # Construct the BIC
        bic = f"{bank_code}{country_code}{location_code}{branch_code}"
        return bic
    
    iban = generate_random_iban(country_code, bank_code_length, account_number_length)
    bic = generate_random_bic()

    transaction['iban'] = iban
    transaction['bic'] = bic

    return transaction



comp_bank_details = comp.create_component_from_func(
    bank_details
)

In [18]:
def card_details(transaction:dict) -> dict:
    from faker import Faker
    import random

    fake = Faker()
    transaction['card_number'] = fake.credit_card_number()
    transaction['card_type'] = random.choice(['Visa', 'MasterCard', 'American Express', 'Discover', 'JCB'])

    return transaction

comp_card_details = comp.create_component_from_func(
    card_details,
    packages_to_install=['faker']
)

## Send to backend

In [19]:
def send_transaction(transaction: dict, host_ip: str, port: int):
    import time
    import requests

    def wait_for_port(host: str, port: int, timeout: int = 60) -> bool:
        """
        Wait until the port on the host is open or the timeout is reached.
        """
        import socket

        start_time = time.time()
        while time.time() - start_time < timeout:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
                result = sock.connect_ex((host, port))
                if result == 0:
                    return True
                time.sleep(1)
        return False


    """
    Waits until the service port on the host machine is open and then sends a POST request.
    """
    api_url = f'http://{host_ip}:{port}'
    
    # Wait until the port is available
    if wait_for_port(host_ip, port):
        print(f"Port {port} on host {host_ip} is open. Sending POST request...")
        
        try:
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.post(api_url, json=transaction, headers=headers)
            response.raise_for_status()
            print(f'Payment successful! Response: {response.json()}')
        except requests.exceptions.RequestException as e:
            print(f'Error: {e}')
    else:
        print(f"Port {port} on host {host_ip} is not open after waiting.")

comp_send_transaction = comp.create_component_from_func(
    send_transaction,
    output_component_file='send_transaction.yaml',
    packages_to_install=['requests']
)

# Pipeline

In [20]:
@dsl.pipeline(
    name='transaction pipeline'
)
def pipeline():
    transaction = comp_transaction_details('2000-01-01', '2024-01-01')
    business_names = comp_business_names()
    parties_involved = comp_parties_involved(transaction.output, business_names.output)
    
    payment_method = comp_extract_payment_method(parties_involved.output)
    
    with dsl.Condition(payment_method.output == 'CARD'):
        card_details = comp_card_details(parties_involved.output)
        send_card_transaction = comp_send_transaction(card_details.output, '192.168.76.119', 3000)
    
    with dsl.Condition(payment_method.output == 'TRANSFER'):
        bank_details = comp_bank_details(parties_involved.output, 'EE')
        send_bank_transaction = comp_send_transaction(bank_details.output, '192.168.76.119', 3000)

Compiler().compile(pipeline, 'transaction_pipeline.yaml')